# FOOTPRINTS

In [1]:
import numpy as np
import datetime
from datetime import date
import json
import pylab
import pandas as pd
import matplotlib.pyplot as plt
import os, sys
from sklearn.preprocessing import normalize

In [2]:

def leer_data():
    outfile='./SOURCES/data.csv'
    data = pd.read_csv(outfile)
    return data

data_original = leer_data()
data = data_original[['client_id','date','año','mes','dia','hora','mccg','quantity','amount_sol']]
data.head(2)

,client_id,date,año,mes,dia,hora,mccg,quantity,amount_sol
0,NNeQwQy9MAQ=,2016-07-15 22:23:25,2016,7,15,22,2,1,100.0
1,qFRoPHPOc/I=,2016-06-21 21:30:55,2016,6,21,21,2,1,20.0


In [3]:
mccgs = ((data.groupby(['mccg'], as_index=False, sort=True)).count())['mccg']
mccgs,mccgs[3],len(mccgs)

(0      2
 1      3
 2      4
 3      5
 4      6
 5      8
 6      9
 7     10
 8     11
 9     12
 10    13
 11    14
 12    15
 13    16
 14    17
 15    18
 16    19
 17    21
 18    22
 19    23
 20    25
 21    26
 22    27
 23    28
 Name: mccg, dtype: int64, 5, 24)

# RESULTADOS

## Unimos los resultados por cada mccg

In [5]:

path='./RESULTS' 

file='./RESULTS/U' 
footprint="%s.footprint" %(file)
individual_clusters="%s.individual_footprint.clusters" %(file)
individual_labels="%s.individual_footprint.labels" %(file)   
collective_clusters="%s.collective_footprint.clusters" %(file)
collective_labels="%s.collective_footprint.labels" %(file)
    
# Unimos
footprint = pd.read_csv(footprint, sep=",", header=0)
footprint.rename(columns={'customer_id':'customer_tag'}, inplace=True)     
individual_labels = pd.read_csv(individual_labels, sep=";", header=0, index_col=False)
collective_labels = pd.read_csv(collective_labels, sep=";", header=0, index_col=False)

In [8]:
footprint.head(5)

,customer_tag,year,week,profile_id,mccg,turn,size,m2t0d0,m2t0d1,m2t0d2,...,m28t2d4,m28t2d5,m28t2d6,m28t3d0,m28t3d1,m28t3d2,m28t3d3,m28t3d4,m28t3d5,m28t3d6
0,+++g8j9k+5A=,2016,27,0,8,0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,++/oQ9Lb9dI=,2017,22,0,25,3,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,++14g8obpj0=,2016,36,0,15,3,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,++14g8obpj0=,2016,35,1,18,0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,++14g8obpj0=,2016,31,2,22,0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
result = pd.merge(individual_labels, collective_labels, on=['customer_tag', 'individual_cluster'])

KeyboardInterrupt: 

In [ ]:
result = pd.merge(individual_labels, collective_labels, on=['customer_tag', 'individual_cluster'])
result = result.iloc[:, :(-1*4*7*len(mccgs))]
result= pd.merge(footprint, result, on=['customer_tag','year', 'week'])
result1 = result.iloc[:, :(-1*4*7*len(mccgs)-1)]
result = pd.concat([result1, result[['collective_cluster']]], axis=1)
result[['year','week','profile_id','size','individual_cluster','collective_cluster']] = result[['year','week','profile_id','size','individual_cluster','collective_cluster']].astype(int)

path_res='%s/Results.csv' %(path)
result.to_csv(path_res,index=False)
print('Done')